There i parse telegram chat history 

In [16]:
!pip install lxml

     ---------------------------------------- 3.6/3.6 MB 1.7 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\royta\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import os
import shutil
from lxml import html
from contextlib import suppress

import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1


import utils

In [2]:
def get_name(body: html.HtmlElement) -> str|None:
    with suppress(Exception):
        name = body.find_class('from_name')[0].text
        name = name.replace('\n', '').strip()
        return name

In [3]:
def get_photo(body: html.HtmlElement) -> str|None:
    with suppress(Exception):
        div = body.find_class('media_wrap')[0]
        a = div.getchildren()[0]
        photo = a.attrib['href']
        return photo

In [4]:
def get_text(body: html.HtmlElement, text="") -> str|None:
    with suppress(Exception):
        div = body.find_class('text')[0]
        for t in div.itertext():
            text+= t
            
        text = text.replace('\n', '', 2).strip()
        return text

In [5]:
def process_photo(photo_path:str, saved_path:str):
    save_embs = True
    if photo_path in embeder.path_id:
        uid = embeder.path_id[photo_path]
        if os.path.exists(f'{saved_path}/{uid}.png'):
            return
        
        else:
            save_embs = False

    img = Image.open(photo_path)
    _, e = photo_path.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path='tmp.png')
    
    # Check for founded face
    if img_cropped is not None:
        img_cropped = img_cropped.unsqueeze(0)
    
        if save_embs:
            embs = [model(img_cropped) for model in embeds_models]
            embeder[photo_path] = embs

        uid = embeder.path_id[photo_path]
        shutil.move('tmp.png', f'{saved_path}/{uid}.png')

In [6]:
def procces_markup(src:str, dst:str):
    if not src.endswith('/'):
        src+= '/'
        
    if dst.endswith('/'):
        dst = dst[:-1]
        
    for file in os.listdir(src):
        photo_path = src + file
        process_photo(photo_path, dst)

In [7]:
mtcnn = MTCNN()
model1 = InceptionResnetV1(pretrained='vggface2').eval()
model2 = InceptionResnetV1(pretrained='casia-webface').eval()

embeds_models = [model1, model2]
embeder = utils.PhotoEmbedingStorage()

In [8]:
messages = []
for catalog in os.listdir('tg_data'):
    
    for file in os.listdir('tg_data/' + catalog):
        if '.html' in file:

            with open(f"tg_data/{catalog}/{file}", "r", encoding='utf-8') as f:
                _html = f.read()

            tree = html.fromstring(_html)
            history = tree.xpath("/html/body/div/div[2]/div")[0]
            history = history.find_class('message default clearfix')

            for message in history:
                body = message.find_class('body')[0]
                    
                photo = get_photo(body) 
                photo = f"{catalog}/{photo}" if photo else None
                messages+= [{
                    'name': get_name(body),
                    'text': get_text(body),
                    'photo': photo,
                }]

In [10]:
with torch.no_grad():
    for current, previous in zip(messages[1:], messages[:-1]):
        if previous['photo']:
            photo_path = 'tg_data/' + previous['photo']

            if current['text'] in ['💌', '❤️', '👍']:
                saved_path = 'data/target'

            elif current['text'] in ['👎']:
                saved_path = 'data/opposite'

            else:
                continue
        
            
            process_photo(photo_path, saved_path)

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [11]:
with torch.no_grad():
    procces_markup('markup/test_target',  "data/test_target/")
    procces_markup('markup/test_opposite',  "data/test_opposite/")
    procces_markup('markup/target',  "data/markup_target/")
    procces_markup('markup/opposite',  "data/markup_opposite/")

In [12]:
embeder.save('emb storage.pkl')

Manually clear the dataset